# Wrangle Exercises

## Data Acquisition

This exercises uses the `case.csv`, `dept.csv`, and `source.csv` files from the san antonio 311 call dataset.

## 1. Read the case, department, and source data into their own spark dataframes.

In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *

spark = SparkSession.builder.getOrCreate()

In [2]:
df_source = spark.read.csv("source.csv", sep = ',', header=True, inferSchema=True)
df_source.show(3)

+---------+----------------+
|source_id| source_username|
+---------+----------------+
|   100137|Merlene Blodgett|
|   103582|     Carmen Cura|
|   106463| Richard Sanchez|
+---------+----------------+
only showing top 3 rows



In [3]:
df = spark.read.csv("case.csv", sep = ',', header=True, inferSchema=True)
df.show(1, vertical=True)

-RECORD 0------------------------------------
 case_id              | 1014127332           
 case_opened_date     | 1/1/18 0:42          
 case_closed_date     | 1/1/18 12:29         
 SLA_due_date         | 9/26/20 0:42         
 case_late            | NO                   
 num_days_late        | -998.5087616000001   
 case_closed          | YES                  
 dept_division        | Field Operations     
 service_request_type | Stray Animal         
 SLA_days             | 999.0                
 case_status          | Closed               
 source_id            | svcCRMLS             
 request_address      | 2315  EL PASO ST,... 
 council_district     | 5                    
only showing top 1 row



In [4]:
dept = spark.read.csv("dept.csv", sep = ',', header=True, inferSchema=True)
dept.show(1, vertical=True)

-RECORD 0----------------------------------
 dept_division          | 311 Call Center  
 dept_name              | Customer Service 
 standardized_dept_name | Customer Service 
 dept_subject_to_SLA    | YES              
only showing top 1 row



## 2. Let's see how writing to the local disk works in spark:

* Write the code necessary to store the source data in both csv and json format, store these as `sources_csv` and `sources_json`
* Inspect your folder structure. What do you notice?

In [5]:
df_source.write.json("souces_json", mode="overwrite")

## 3. Inspect the data in your dataframes. 

* Are the data types appropriate?
* Write the code necessary to cast the values to the appropriate types.

In [6]:
# Turn values to bools
df = df.withColumn("case_closed", expr('case_closed == "YES"')).withColumn(
    "case_late", expr('case_late == "YES"')
)

In [7]:
# Turn district to string
df = df.withColumn("council_district", col("council_district").cast("string"))

In [8]:
# Edit format of the address
df = df.withColumn("request_address", trim(lower(df.request_address)))

In [9]:
# turn days late to weeks 
df = df.withColumn(
    "num_weeks_late", expr("num_days_late / 7 AS num_weeks_late")
)

In [10]:
# change dates to timestamp
fmt = "M/d/yy H:mm"
df = (
    df.withColumn("case_opened_date", to_timestamp("case_opened_date", fmt))
    .withColumn("case_closed_date", to_timestamp("case_opened_date", fmt))
    .withColumn("SLA_due_date", to_timestamp("case_opened_date", fmt))
)

In [11]:
df = (
    df.withColumn(
        "case_age", datediff(current_timestamp(), "case_opened_date")
    )
    .withColumn(
        "days_to_closed", datediff("case_closed_date", "case_opened_date")
    )
    .withColumn(
        "case_lifetime",
        when(expr("! case_closed"), col("case_age")).otherwise(
            col("days_to_closed")
        ),
    )
)

df.select(
    "case_closed",
    "case_opened_date",
    "case_closed_date",
    "case_age",
    "days_to_closed",
    "case_lifetime",
).where(expr("case_closed")).show(5)

df.select(
    "case_closed",
    "case_opened_date",
    "case_closed_date",
    "case_age",
    "days_to_closed",
    "case_lifetime",
).where(expr("! case_closed")).show(5)

+-----------+-------------------+-------------------+--------+--------------+-------------+
|case_closed|   case_opened_date|   case_closed_date|case_age|days_to_closed|case_lifetime|
+-----------+-------------------+-------------------+--------+--------------+-------------+
|       true|2018-01-01 00:42:00|2018-01-01 00:42:00|     889|             0|            0|
|       true|2018-01-01 00:46:00|2018-01-01 00:46:00|     889|             0|            0|
|       true|2018-01-01 00:48:00|2018-01-01 00:48:00|     889|             0|            0|
|       true|2018-01-01 01:29:00|2018-01-01 01:29:00|     889|             0|            0|
|       true|2018-01-01 01:34:00|2018-01-01 01:34:00|     889|             0|            0|
+-----------+-------------------+-------------------+--------+--------------+-------------+
only showing top 5 rows

+-----------+-------------------+-------------------+--------+--------------+-------------+
|case_closed|   case_opened_date|   case_closed_date|ca

In [12]:
df = (
    df
    # left join on dept_division
    .join(dept, "dept_division", "left")
    # drop all the columns except for standardized name, as it has much fewer unique values
    .drop(dept.dept_division)
    .drop(dept.dept_name)
    .withColumnRenamed("standardized_dept_name", "department")
    # convert to a boolean
    .withColumn("dept_subject_to_SLA", col("dept_subject_to_SLA") == "YES")
)

-------

# Prepare Exercises

## 1. How old is the latest (in terms of days past SLA) currently open issue? How long has the oldest (in terms of days since opened) currently opened issue been open?

In [13]:
df.show(1, vertical=True)

-RECORD 0------------------------------------
 dept_division        | Field Operations     
 case_id              | 1014127332           
 case_opened_date     | 2018-01-01 00:42:00  
 case_closed_date     | 2018-01-01 00:42:00  
 SLA_due_date         | 2018-01-01 00:42:00  
 case_late            | false                
 num_days_late        | -998.5087616000001   
 case_closed          | true                 
 service_request_type | Stray Animal         
 SLA_days             | 999.0                
 case_status          | Closed               
 source_id            | svcCRMLS             
 request_address      | 2315  el paso st,... 
 council_district     | 5                    
 num_weeks_late       | -142.6441088         
 case_age             | 889                  
 days_to_closed       | 0                    
 case_lifetime        | 0                    
 department           | Animal Care Services 
 dept_subject_to_SLA  | true                 
only showing top 1 row



In [14]:
(
    df.select(df.SLA_due_date, df.case_opened_date, df.case_closed_date, df.case_closed)
    .filter(df.case_closed == "false")
    .sort("case_opened_date")
).first()

Row(SLA_due_date=datetime.datetime(2017, 1, 1, 13, 48), case_opened_date=datetime.datetime(2017, 1, 1, 13, 48), case_closed_date=datetime.datetime(2017, 1, 1, 13, 48), case_closed=False)

## 2. How many Stray Animal cases are there?

In [15]:
df.filter(df.service_request_type == "Stray Animal").count()

26760

## 3. How many service requests that are assigned to the Field Operations department (`dept_division`) are not classified as "Officer Standby" request type (`service_request_type`)?

In [18]:
(
    df.filter(df.dept_division == "Field Operations")
    .filter(df.service_request_type != "Officer Standby")
).count()

113902

## 4. Convert the council_district column to a string column.

In [19]:
# Done in the acquire section

## 5. Extract the year from the case_closed_date column.

In [26]:
(
    df.withColumn("year", year("case_closed_date"))
    .select("year")
    .show()
)

+----+
|year|
+----+
|2018|
|2018|
|2018|
|2018|
|2018|
|2018|
|2018|
|2018|
|2018|
|2018|
|2018|
|2018|
|2018|
|2018|
|2018|
|2018|
|2018|
|2018|
|2018|
|2018|
+----+
only showing top 20 rows



## 6. Convert num_days_late from days to hours in new columns num_hours_late.

In [27]:
# turn days late to weeks 
df = df.withColumn(
    "num_hours_late", expr("num_days_late * 24 AS num_hours_late")
)

In [28]:
df.show(1, vertical=True)

-RECORD 0------------------------------------
 dept_division        | Field Operations     
 case_id              | 1014127332           
 case_opened_date     | 2018-01-01 00:42:00  
 case_closed_date     | 2018-01-01 00:42:00  
 SLA_due_date         | 2018-01-01 00:42:00  
 case_late            | false                
 num_days_late        | -998.5087616000001   
 case_closed          | true                 
 service_request_type | Stray Animal         
 SLA_days             | 999.0                
 case_status          | Closed               
 source_id            | svcCRMLS             
 request_address      | 2315  el paso st,... 
 council_district     | 5                    
 num_weeks_late       | -142.6441088         
 case_age             | 889                  
 days_to_closed       | 0                    
 case_lifetime        | 0                    
 department           | Animal Care Services 
 dept_subject_to_SLA  | true                 
 num_hours_late       | -23964.210

## 7. Join the case data with the source and department data.

In [30]:
df_source.show(1)

+---------+----------------+
|source_id| source_username|
+---------+----------------+
|   100137|Merlene Blodgett|
+---------+----------------+
only showing top 1 row



In [34]:
df = (
    df
    # left join on dept_division
    .join(df_source, "source_id", "left")
)

In [36]:
df.show(3, vertical=True)

-RECORD 0------------------------------------
 source_id            | svcCRMLS             
 dept_division        | Field Operations     
 case_id              | 1014127332           
 case_opened_date     | 2018-01-01 00:42:00  
 case_closed_date     | 2018-01-01 00:42:00  
 SLA_due_date         | 2018-01-01 00:42:00  
 case_late            | false                
 num_days_late        | -998.5087616000001   
 case_closed          | true                 
 service_request_type | Stray Animal         
 SLA_days             | 999.0                
 case_status          | Closed               
 request_address      | 2315  el paso st,... 
 council_district     | 5                    
 num_weeks_late       | -142.6441088         
 case_age             | 889                  
 days_to_closed       | 0                    
 case_lifetime        | 0                    
 department           | Animal Care Services 
 dept_subject_to_SLA  | true                 
 num_hours_late       | -23964.210